<a href="https://colab.research.google.com/github/arangoml/arangopipe/blob/master/examples/Arangopipe_Feature_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation Prerequisites


In [ ]:
!pip install python-arango
!pip install arangopipe==0.0.6.9.3
!pip install pandas PyYAML==5.1.1 sklearn2
!pip install jsonpickle

# Arangopipe for Machine Learning Metadata Management

Machine learning model development requires extensive experimentation. Experiments are required for:
1. Understanding the characteristics of the data
2. Testing hypothesis about the data
3. Testing candidate models for a machine learning task.
4. Determining the best set of hyper-parameters for a machine learning task.
5. Determining the best optimization algorithm for a machine learning task
6. Determining the best parameters for an optimization algorithm.
7. Determine the effects of various pre-procrossesing tasks on algorithm performance
8. Feature selection and feature extraction.
9. Best loss function for a machine learning task
10. Best model metric to match a business metric.


The above list is by no means exhaustive. They are just some of the motivations for performing experiments for machine learning model development. When these experiments are performed, there is a need to capture the results of the experiments and key results in the form of standard machine learning artifacts, for example a notebook used for performing this experiment. There is also a need to organize the data captured from the above experiments in a cohesive manner. Related project activities must be connected. Personel working of machine learning projects must be able to query this data in an adhoc manner to obtain information that they or their co-workers had captured. This is the need for which Arangopipe was created.

Arangopipe is a tool for machine learning metadata management using ArangoDB. ArangoDB is a multi-model database and supports a graph and document oriented data model. This makes it a very convinient tool to store meta-data from machine learning experiments. Arangopipe is an API for caturing and analyzing meta-data from machine learning experiments. In this notebook we provide a brief overview of tracking machine learning metadata using Arangopipe. For more information about Arangopipe, please see the [project](https://www.arangodb.com/machine-learning/) page.

**NOTE:**


You should be able to run this notebook without installing ArangoDB. The notebook connects to a managed service (cloud) instance of ArangoDB.

## Creating a Project

To use Arangopipe to track meta-data for projects, projects have to be registered with Arangopipe. For purposes of illustration, we will use the california housing dataset from UCI machine learning repository. Our project entails developing a regression model with this dataset. We will first register this project with Arangopipe as shown below.


In [ ]:

from arangopipe.arangopipe_storage.arangopipe_api import ArangoPipe
from arangopipe.arangopipe_storage.arangopipe_admin_api import ArangoPipeAdmin
from arangopipe.arangopipe_storage.arangopipe_config import ArangoPipeConfig
from arangopipe.arangopipe_storage.managed_service_conn_parameters import ManagedServiceConnParam
mdb_config = ArangoPipeConfig()
msc = ManagedServiceConnParam()
conn_params = { msc.DB_SERVICE_HOST : "arangoml.arangodb.cloud", \
                        msc.DB_SERVICE_END_POINT : "createDB",\
                        msc.DB_SERVICE_NAME : "createDB",\
                        msc.DB_SERVICE_PORT : 8529,\
                        msc.DB_CONN_PROTOCOL : 'https'}
        
mdb_config = mdb_config.create_connection_config(conn_params)
admin = ArangoPipeAdmin(reuse_connection = False, config = mdb_config)
ap_config = admin.get_config()
ap = ArangoPipe(config = ap_config)
proj_info = {"name": "Housing_Price_Estimation_Project"}
proj_reg = admin.register_project(proj_info)

## Model Building


In this section, the details of capturing meta-data with Arangopipe as part of model building activity will be illustrated. Model selection is an important activity for data scientists. Data scientists consider many candidate models for a task. The best performing model is then chosen. This procedure is illustrated in the notebook illustrating the use of hyperopt to capture meta data from a hyper-parameter tuning experiment, (see [hyperopt.](https://github.com/arangoml/arangopipe/blob/master/arangopipe/tests/hyperopt/hyperopt_integration.ipynb)). We will use a simpler setting for this notebook. We will assume model selection has been completed and that a LASSO regression model is the best candidate for the task. Having made this decision, we capture information about the model and its parameters. This information is stored in Arangopipe. The details of performing these tasks are shown below. Before model building, we capture information related to the dataset and the features used to build the model.


### Register Dataset

Read a copy of the dataset from the Arangopipe repository

In [ ]:
import pandas as pd
data_url = "https://raw.githubusercontent.com/arangoml/arangopipe/arangopipe_examples/examples/data/cal_housing.csv"
df = pd.read_csv(data_url, error_bad_lines=False)

In [ ]:
df.head()

Register it with Arangopipe.

In [ ]:

ds_info = {"name" : "california-housing-dataset",\
            "description": "This dataset lists median house prices in Califoria. Various house features are provided",\
           "source": "UCI ML Repository" }
ds_reg = ap.register_dataset(ds_info)

### Register Featureset

Register the features used to develop the model.


*   Note that the response variable has been log transformed
*   Note that when the featureset is registered, it is linked to the dataset



In [ ]:
import numpy as np
df["medianHouseValue"] = df["medianHouseValue"].apply(lambda x: np.log(x))
featureset = df.dtypes.to_dict()
featureset = {k:str(featureset[k]) for k in featureset}
featureset["name"] = "log_transformed_median_house_value"
fs_reg = ap.register_featureset(featureset, ds_reg["_key"]) # note that the dataset and featureset are linked here.

### Generate Data Summaries

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.hist()

### Develop a Model

Create test and training sets for the model building activity

In [ ]:
from sklearn.model_selection import train_test_split
preds = df.columns.to_list()
preds.remove('medianHouseValue')
X = df[preds].values
Y = df['medianHouseValue'].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

Fit a Lasso model

A Lasso model is developed. Train and test performances are noted.

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
clf = linear_model.Lasso(alpha=0.001)
clf.fit(X_train, y_train)
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)
train_mse = mean_squared_error(train_pred, y_train)
test_mse = mean_squared_error(test_pred, y_test)


### Register the Model
* Note that project and model are linked
* The notebook associated withe the model can be retreived from github. This can be part of the meta-data associated with the model


In [ ]:
import io
import requests
url = ('https://raw.githubusercontent.com/arangoml/arangopipe/master/examples/Arangopipe_Feature_Examples.ipynb')
nbjson = requests.get(url).text

In [ ]:

model_info = {"name": "Lasso Model for Housing Dataset",  "task": "Regression", 'notebook': nbjson}
model_reg = ap.register_model(model_info, project = "Housing_Price_Estimation_Project")


## Log Model Building Activity

In this section the details of capturing a consolidated version of this model building activity. The execution of this notebook, or any ML activity, is captured by the 'Run' entity in the Arangopipe schema (see [schema](https://github.com/arangoml/arangopipe)). To record the execution, we need to create a unique identifier for it in ArangoDB. After generating a unique identifier, we capture the model parameters and model performance and then record the details of this experiment in Arangopipe. Each of these steps is shown below.

Note: Model parameters are important metadata. Results from model building activity are converted to JSON for storage in ArangoDB. We are now getting ready to create a consolidated capture of the activities performed in this notebook in Arangopipe. To do so, we need to create a unique identifier for this activity. These are shown below.



Note that capturing the 'Run' or execution of this cell captures information that links


1.   The dataset used in this execution (ds_reg)
2.   The featureset used in this execution (fs_reg)
3.   The model parameters used in this execution (model_params)
4.   The model performance that was observed in this execution (model perf)



In [ ]:
import uuid
import datetime
import jsonpickle

ruuid = str(uuid.uuid4().int)
model_perf = {'training_mse': train_mse, 'test_mse': test_mse, 'run_id': ruuid, "timestamp": str(datetime.datetime.now())}

mp = clf.get_params()
mp = jsonpickle.encode(mp)
model_params = {'run_id': ruuid, 'model_params': mp}

run_info = {"dataset" : ds_reg["_key"],\
                    "featureset": fs_reg["_key"],\
                    "run_id": ruuid,\
                    "model": model_reg["_key"],\
                    "model-params": model_params,\
                    "model-perf": model_perf,\
                    "tag": "Housing_Price_Estimation_Project",\
                    "project": "Housing_Price_Estimation_Project"}
ap.log_run(run_info)

### Save the connection information to google drive so that this can used to connect to the instance that was used in this session,

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
fp = '/content/drive/My Drive/saved_arangopipe_config.yaml'
mdb_config.export_cfg(fp)

## Using Arangopipe with Common Tools in a Machine Learning Stack

This notebook provides the details of working with Arangopipe to capture meta-data from a machine learning project activity. If you would like to see Arangopipe can be used with some common tools in a machine learning stack:


1.   See [TFX](https://github.com/arangoml/arangopipe/tree/master/arangopipe/tests/TFX) for the details of using Arangopipe with TFX
2.   See [Pytorch](https://github.com/arangoml/arangopipe/tree/master/arangopipe/tests/pytorch) for details of using Arangopipe with Pytorch.
3.  See [Hyperopt](https://github.com/arangoml/arangopipe/tree/master/arangopipe/tests/hyperopt) for details of using Arangopipe with Hyperopt
4. See [MLFlow](https://github.com/arangoml/arangopipe/tree/master/arangopipe/tests/mlflow) for details of using Arangopipe with MLFlow.

